In [1]:
import json
import pandas
from collections import Counter, defaultdict
import pickle

In [2]:
%%bash
#scp filten@lod.labs.vu.nl:/home/filten/LongTailQATask/QuestionCreation/test_output/*.json debug
#scp filten@lod.labs.vu.nl:/home/filten/LongTailQATask/QuestionCreation/q_id2docs.json debug

In [3]:
q_id2docs = pickle.load(open('debug/q_id2docs.json', 'rb'))

In [8]:
headers = ['subtask', 'confusion', 'event_type', '#Qs', 'Answer', 'Avg_gold_docs', 'Zero_answer', 'Avg_c2a_ratio']
list_of_lists = []

freqs = dict()

for subtask in ['1', '2', '3']:
    
    answers = json.load(open(f'debug/{subtask}_answers.json'))
    questions = json.load(open(f'debug/{subtask}_questions.json'))
    num_incidents = []
    num_gold_docs = []
    for q_id, answers_info in answers.items():
        
        q_info = questions[q_id]
        
        confusion_settings = []
        if 'participant' in q_info:
            confusion_settings.append('part')
        if 'time' in q_info:
            confusion_settings.append('time')
        if 'location' in q_info:
            confusion_settings.append('loc')

        setting = '-'.join(confusion_settings)
        event_type = q_info['event_types'][0]
        
        if (subtask, setting, event_type) not in freqs:
            freqs[(subtask, setting, event_type)] = {'answers' : [], 
                                                     'num_gold_docs' : [],
                                                     'c2a_ratio' : []}
            
        the_gold_docs = {gold_doc
                         for gold_docs in answers_info['answer_docs'].values()
                         for gold_doc in gold_docs}
        
        all_docs = q_id2docs[f'test_output/system_input/{q_id}']
        confusion_docs = all_docs - the_gold_docs

        if len(the_gold_docs) == 0:
            c2a_ratio = 0.0
        else:
            c2a_ratio = len(confusion_docs) / len(the_gold_docs)
        
        
        freqs[(subtask, setting, event_type)]['num_gold_docs'].append(len(the_gold_docs))
        freqs[(subtask, setting, event_type)]['answers'].append(answers_info['numerical_answer'])
        freqs[(subtask, setting, event_type)]['c2a_ratio'].append(c2a_ratio)

        
        
        if (subtask, 'all', 'all') not in freqs:
            freqs[(subtask, 'all', 'all')] = {'answers' : [], 
                                              'num_gold_docs' : [],
                                              'c2a_ratio' : []}
            
            
        freqs[(subtask, 'all', 'all')]['num_gold_docs'].append(len(the_gold_docs))
        freqs[(subtask, 'all', 'all')]['answers'].append(answers_info['numerical_answer'])
        freqs[(subtask, 'all', 'all')]['c2a_ratio'].append(c2a_ratio)

for (subtask, setting, event_type), info in freqs.items():
    
    avg_num_incidents = sum(info['answers']) / len(info['answers'])
    avg_num_gold_docs = sum(info['num_gold_docs']) / len(info['num_gold_docs'])
    avg_c2a_ratio = sum(info['c2a_ratio']) / len(info['c2a_ratio'])


    export = [subtask, 
              setting,
              event_type,
              len(info['answers']), 
              'min:%s avg:%s max:%s' % (min(info['answers']), round(avg_num_incidents, 2), max(info['answers']) ),
              round(avg_num_gold_docs, 2),
              info['answers'].count(0),
              round(avg_c2a_ratio, 2)]
    list_of_lists.append(export)
    
stats_df = pandas.DataFrame(list_of_lists, columns=headers)
stats_df

,subtask,confusion,event_type,#Qs,Answer,Avg_gold_docs,Zero_answer,Avg_c2a_ratio
0,1,time-loc,killing,100,min:1 avg:1.0 max:1,1.59,0,152.11
1,1,all,all,1016,min:1 avg:1.0 max:1,1.68,0,153.64
2,1,part-time,killing,144,min:1 avg:1.0 max:1,2.12,0,237.85
3,1,part-loc,killing,75,min:1 avg:1.0 max:1,2.01,0,38.12
4,1,time-loc,injuring,527,min:1 avg:1.0 max:1,1.46,0,141.70
5,1,part-time,injuring,108,min:1 avg:1.0 max:1,1.86,0,238.64
6,1,part-loc,injuring,62,min:1 avg:1.0 max:1,2.00,0,53.61
7,2,part-time,killing,197,min:0 avg:2.33 max:11,5.22,14,235.24
8,2,all,all,1195,min:0 avg:3.77 max:67,6.96,50,145.05
9,2,time-loc,injuring,518,min:0 avg:4.69 max:67,7.73,4,113.85


## granularities

In [ ]:
#headers = ['subtask', 'first', 'event_type', '#Qs', 'Answer', 'Avg_gold_docs', 'Zero_answer']
list_of_lists = []

freqs = {'1': defaultdict(int),
         '2': defaultdict(int),
         '3': defaultdict(int)}

for subtask in ['1', '2', '3']:
    
    answers = json.load(open(f'debug/{subtask}_answers.json'))
    questions = json.load(open(f'debug/{subtask}_questions.json'))
    for q_id, answers_info in answers.items():
        
        q_info = questions[q_id]
        
        for key in ['location', 'time', 'participant']:
            if key in q_info:
                gran_level = list(q_info[key])[0]
                freqs[subtask][gran_level] += 1

        
freqs